In [ ]:
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Função para criar o modelo
def create_model(input_dim, params):
    model = Sequential([
        Dense(units=params['units_1'], input_dim=input_dim, activation='relu'),
        Dense(units=params['units_2'], activation='relu'),

        Dropout(params['dropout']),

        Dense(units=params['units_3'], activation='relu'),
        Dense(units=params['units_4'], activation='relu'),

        Dense(units=1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def fitness_function(params):
    model = create_model(params['X_train'].shape[1], params)
    model.fit(params['X_train'], params['y_train'], epochs=5, batch_size=15, verbose=1, validation_data=(params['X_validate'], params['y_validate']))
    predictions = model.predict(params['X_test'])
    predictions = np.round(predictions).astype(int)
    accuracy = accuracy_score(params['y_test'], predictions)
    return accuracy

def create_individual(X, y):
    X_train, X_split, y_train, y_split = train_test_split(X, y, test_size=0.4, random_state=0)
    X_test, X_validate, y_test, y_validate = train_test_split(X_split, y_split, test_size=0.5, random_state=0)

    return {
        'units_1': random.randint(8, 64),
        'units_2': random.randint(8, 64),
        'dropout': random.uniform(0.2, 0.5),
        'units_3': random.randint(8, 64),
        'units_4': random.randint(8, 64),
        'X': X,
        'y': y,
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test,
        'X_validate': X_validate,
        'y_validate': y_validate
    }


def crossover(parent1, parent2):
    child = {}
    for key in parent1.keys():
        if random.random() < 0.5:
            child[key] = parent1[key]
        else:
            child[key] = parent2[key]
    return child


def mutate(individual):
    mutation_rate = 0.1
    if random.random() < mutation_rate:
        individual['units_1'] = random.randint(8, 64)
    if random.random() < mutation_rate:
        individual['units_2'] = random.randint(8, 64)
    if random.random() < mutation_rate:
        individual['dropout'] = random.uniform(0.2, 0.5)
    if random.random() < mutation_rate:
        individual['units_3'] = random.randint(8, 64)
    if random.random() < mutation_rate:
        individual['units_4'] = random.randint(8, 64)
    return individual


def select(population, fitnesses):
    selected = np.random.choice(len(population), size=2, p=fitnesses/np.sum(fitnesses))
    return population[selected[0]], population[selected[1]]


def genetic_algorithm(population, generations, population_size):    
    for i, generation in enumerate(range(generations)):
        print(f"Init generation {generation}:")
        fitnesses = np.array([fitness_function(individual) for individual in population])
        new_population = []
        for _ in range(population_size):
            parent1, parent2 = select(population, fitnesses)
            child = crossover(parent1, parent2)
            child = mutate(child)
            new_population.append(child)
        population = new_population
        best_fitness = np.max(fitnesses)
        print(f"Generation {generation}: Best Fitness = {best_fitness}")

    best_individual = population[np.argmax(fitnesses)]
    return best_individual

In [ ]:
data = pd.read_csv('creditcard.csv')

standardscaler = StandardScaler()
data['normalized_amount'] = standardscaler.fit_transform(data['Amount'].values.reshape(-1, 1))
data['normalized_amount']

data = data.drop(['Amount', 'Time'], axis=1)


y = data['Class']
X = data.drop(['Class'], axis=1)

In [ ]:

population_size=10
population = [create_individual(X, y) for _ in range(population_size)]
best_params = genetic_algorithm(population, generations=10, population_size=population_size)
print("Best parameters found:", best_params)